试验了将sequence example 写入tfrecord中

In [ ]:
#coding=utf-8

import tensorflow as tf
import os
labels=[2,3,5]
keys=[[1,2],[2,5,6],[2,5,6,7,8,9]]

sess=tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

def make_example(label,key):
    
    example = tf.train.SequenceExample(
        context=tf.train.Features(
            feature={
            "label":tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
        }),
        feature_lists=tf.train.FeatureLists(
            feature_list={
            "feature":tf.train.FeatureList(feature=[tf.train.Feature(int64_list=tf.train.Int64List(value=[key[i]])) for i in range(len(key))])
            }
        )
    )
    return example.SerializeToString()


filename="./tmp.tfrecords"
if os.path.exists(filename):
    os.remove(filename)
writer = tf.python_io.TFRecordWriter(filename)
for k in range(len(keys)):
    key=keys[k]
    label=labels[k]
    ex = make_example(label,key)
    writer.write(ex)
writer.close()



主要试验了 sequence example 的contxt 和sequence_features的用法，得出的结论可以将sequence_features在batch内进行padding，但是textcnn是要求在所有样本采用同一个句子长度，因此不行

所以只能在做数据处理时候先给padding 到固定长度之后再写入到tfrecord中了。

In [ ]:
import tensorflow as tf
from tflearn.data_utils import to_categorical, pad_sequences
reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(["./tmp.tfrecords"],num_epochs=3)
_,serialized_example =reader.read(filename_queue)

# coord = tf.train.Coordinator()
# threads = tf.train.start_queue_runners(sess=sess,coord=coord)

context_features={
    "label":tf.FixedLenFeature([],dtype=tf.int64)
}
sequence_features={
    "feature":tf.FixedLenSequenceFeature([],dtype=tf.int64)
}

context_parsed, sequence_parsed = tf.parse_single_sequence_example(
    serialized=serialized_example,
    context_features=context_features,
    sequence_features=sequence_features
)

#batch_data = tf.train.batch(tensors=[sequence_parsed['feature']],batch_size=2,dynamic_pad=True,allow_smaller_final_batch=True)
batch_label,batch_data = tf.train.batch(tensors=[context_parsed['label'], sequence_parsed['feature']],batch_size=2,dynamic_pad=True,allow_smaller_final_batch=True)
# batch_label,batch_data = tf.train.batch(tensors=[context_parsed['label'], sequence_parsed['feature']],batch_size=2,allow_smaller_final_batch=True)
print(batch_data)


print(batch_label)

# result = tf.contrib.learn.run_n({"index":batch_data})
init_op = [
      tf.global_variables_initializer(),
      tf.local_variables_initializer()
  ]
with tf.Session() as sess:
    sess.run(init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord, sess=sess)
    try:
        while not coord.should_stop():
            print(sess.run([batch_label,batch_data]))
    except tf.errors.OutOfRangeError:
         print("Done train")


如下主要是把不定长的key写入到tfrecord中，只不过不是按sequence example的方式

In [ ]:
#coding=utf-8

import tensorflow as tf
import os
labels=[2,3,5]
keys=[[1,2],[2,5,6],[2,5,6,7,8,9]]

sess=tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

def make_example(label,key):
    example = tf.train.Example(features=tf.train.Features(
        feature={
            'label':tf.train.Feature(float_list=tf.train.FloatList(value=[label])),
              'vu':tf.train.Feature(float_list=tf.train.FloatList(value=key))
          
        }
    )
                                )
    return example.SerializeToString()


filename="./tmp4.tfrecords"
if os.path.exists(filename):
    os.remove(filename)
writer = tf.python_io.TFRecordWriter(filename)
for k in range(len(keys)):
    key=keys[k]
    label=labels[k]
    ex = make_example(label,key)
    writer.write(ex)
writer.close()

如下的主要是试验了VarLenFeature的用法，如果是tf.VarLenFeature(tf.float32)，会被解析成SparseTensor；这样是不能满足text cnn的要求的，textcnn 要求是一个句子同一个长度，这样可以解析成句子长度行*embeding的尺寸的列数

如果key=tf.FixedLenFeature([], tf.float32)好像总会报错，因为key本来就不是定长的，按道理不会报错吧？？如果不会就可以满足text cnn 的要求，也即是每个句子的长度固定，不够的用0来补


In [ ]:
import tensorflow as tf
from tflearn.data_utils import to_categorical, pad_sequences
reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(["./tmp4.tfrecords"],num_epochs=3)
_,batch_serialized_example =reader.read(filename_queue)

# coord = tf.train.Coordinator()
# threads = tf.train.start_queue_runners(sess=sess,coord=coord)



features = tf.parse_single_example(
      batch_serialized_example,
      features={
          "label": tf.FixedLenFeature([], tf.float32),
#           "key":tf.VarLenFeature(tf.float32)
           "vu": tf.VarLenFeature(tf.float32)
      })
batch_labels = features["label"]
batch_values = features["vu"]

#batch_data = tf.train.batch(tensors=[sequence_parsed['feature']],batch_size=2,dynamic_pad=True,allow_smaller_final_batch=True)
batch_label,batch_data = tf.train.batch(tensors=[batch_labels, batch_values],
                                        batch_size=2)
# batch_label,batch_data = tf.train.batch(tensors=[context_parsed['label'], sequence_parsed['feature']],batch_size=2,allow_smaller_final_batch=True)

print(batch_data)


print(batch_label)

# result = tf.contrib.learn.run_n({"index":batch_data})
init_op = [
      tf.global_variables_initializer(),
      tf.local_variables_initializer()
  ]
with tf.Session() as sess:
    sess.run(init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord, sess=sess)
    try:
        while not coord.should_stop():
            print(sess.run([batch_label,batch_data]))
    except tf.errors.OutOfRangeError:
         print("Done train")


# embedding_lookup_sparsesparse
这个主要演示了tf.nn.embedding_lookup_sparsesparse，sparse中的向量实际上是[[1	null	2],[2	0	null]],所以计算过程是先根据1对应embeding的向量（也即是a的第2行）[2 3]，然后根据2对应的embedding的向量（也即是a的第3行）[4 5]，然后相加得到sparse第0行的向量，在embedding查询后是[6. 8.]，sparse的第1行也类似

In [ ]:
#coding=utf-8
import numpy as np
import tensorflow as tf
a = np.arange(8).reshape(4, 2)


print(a)
# print(b)
# print(c)

a = tf.Variable(a, dtype=tf.float32)

from tensorflow.python import pywrap_tensorflow
model_dir='C:/work/tensorflow_template/log/model.ckpt'
checkpoint_path =model_dir 
reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path) 
print('22')
var_to_shape_map = reader.get_variable_to_shape_map()
aa=reader.get_tensor('embeddings/Variable')
for key in var_to_shape_map:  
    print("tensor_name: ", key)  
    print(reader.get_tensor(key)) #

idx = tf.SparseTensor(indices=[[0, 0], [0, 2], [1, 0], [1, 1]], values=[1, 2, 2, 0], dense_shape=(2, 3))
result = tf.nn.embedding_lookup_sparse(aa, idx, None, combiner="sum")

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
id,result2 = sess.run([idx,result])
print('---')
print(result2)

In [ ]:
print(id)

In [ ]:
# 

def input_fn(data_file):
  """Generate an input function for the Estimator."""
  assert tf.gfile.Exists(data_file), (
      '%s not found. Please make sure you have either run data_download.py or '
      'set both arguments --train_data and --test_data.' % data_file)

  def parse_csv(value):
    print('Parsing', data_file)
    columns = tf.decode_csv(value, record_defaults=_CSV_COLUMN_DEFAULTS)
    features = dict(zip(_CSV_COLUMNS, columns))
    
    return features

  # Extract lines from input files using the Dataset API.
  dataset = tf.data.TextLineDataset(data_file)
  dataset = dataset.map(parse_csv, num_parallel_calls=1)
  return dataset

In [ ]:
# one_element
import tensorflow as tf
import numpy as np
a=tf.placeholder(tf.float32,shape=(5, None))
w = np.array([[1.0, 2.0, 3.0, 4.0, 5.0]])
w=np.transpose(w)

dataset = tf.data.Dataset.from_tensor_slices(w)
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
with tf.Session() as sess:
    try:
        while True:
            print(sess.run(one_element,feed_dict={a:w}))
    except tf.errors.OutOfRangeError:
        print("end!")

In [ ]:
from tensorflow.python import pywrap_tensorflow
model_dir='C:\work\tensorflow_template\log\checkpoint\model.ckpt'
checkpoint_path =model_dir 
reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path) 
print('22')
var_to_shape_map = reader.get_variable_to_shape_map()  
for key in var_to_shape_map:  
    print("tensor_name: ", key)  
    print(reader.get_tensor(key)) #

In [ ]:
import codecs
with codecs.open('./metadata.tsv', 'w','utf8') as f:
    f.write('நியூஸ்7 தமிழ்')

In [ ]:
import tensorflow as tf

directory =[]
for i in range(1, 555576, 10000):
#     output_filename="C:/work/tensorflow_template2/tags_{}.tfrecords".format(i)
    output_filename="s3://shareit.tmp/tangguoping/tags/tags_{}.tfrecords".format(i)
    directory.append(output_filename)
file_names = tf.train.match_filenames_once(directory)

init = (tf.global_variables_initializer(), tf.local_variables_initializer())

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(file_names))
    
    
for i in range(5):
    print(i)
print(i)

In [ ]:
with codecs.open('./metadata.tsv', 'r','utf8') as f:
  all_line_txt = f.readlines()
  for (i,l) in enumerate(all_line_txt):
    print(l)
import csv
csv_reader = csv.reader(open('./metadata.tsv', encoding='utf-8'))
for row in csv_reader:
        print(row[0])

In [ ]:
import tensorflow as tf 

# 新建一个Session
with tf.Session() as sess:
    # 我们要读三幅图片A.jpg, B.jpg, C.jpg
    filename = ['A.jpg', 'B.jpg', 'C.jpg']
    # string_input_producer会产生一个文件名队列
    filename_queue = tf.train.string_input_producer(filename, shuffle=False, num_epochs=5)
  
    # tf.train.string_input_producer定义了一个epoch变量，要对它进行初始化
    tf.local_variables_initializer().run()
    # 使用start_queue_runners之后，才会开始填充队列
    threads = tf.train.start_queue_runners(sess=sess)
    i = 0
    while True:
        i += 1
        # 获取图片数据并保存
        print(sess.run(filename_queue))

In [ ]:
import tensorflow as tf
import numpy as np

c=tf.constant(value=1)
print( c.graph ==tf.get_default_graph())
print(c.graph)
print(tf.get_default_graph())

# graph的含义，在with里面写的ops都属于那个图，不在的属于默认图

In [ ]:
import tensorflow as tf
import numpy as np

c=tf.constant(value=1)
print(c.graph == tf.get_default_graph())
print(c.graph)
print(tf.get_default_graph())

g=tf.Graph()
print("g:",g)
with g.as_default():
    d=tf.constant(value=2)
    print("d:",d.graph)
    #print(g)

g2=tf.Graph()
print("g2:",g2)
g2.as_default()
e=tf.constant(value=15)
print("e:",e.graph)

上面的例子里面创创建了一个新的图g，然后把g设为默认，那么接下来的操作不是在默认的图中，而是在g中了。你也可以认为现在g这个图就是新的默认的图了。
要注意的是，最后一个量e不是定义在with语句里面的，也就是说，e会包含在最开始的那个图中。也就是说，要在某个graph里面定义量，要在with语句的范围里面定义。

## Save & restore & continue train in tensorflow

1.globalstep的主要作用是，例如想 多次训练模型，那么就有local step(指的是本次训练在哪一步)，和global_stepI（在所有训练中这个有属于哪一步）,globalstep在minimize时候更新:

train = optimizer.minimize(loss,global_step=global_step)  

2. saver.restore之后就可以用同一套代码接着训练了。这段demo，分2次运行，可以看到第一次w 从-0.5219252,b从0开始，结尾是w=5.7128973，b=8.272915，运行时候第二次w=5.7128973，b=8.272915开始因此，这个就是从第一次的结尾继续训练。

global_step第二次变为9开始，因此这个也是从第一次接着往下，那么在summary 画图时候就额可以拼接多次训练的成果了。

此外发现W = tf.Variable(tf.random_uniform([1], -1.0, 1.0)) ，加入seed=1之后，第二次还是从-0.5219252，奇怪。

In [1]:
import tensorflow as tf
import numpy as np
# 模拟生成100对数据对, 对应的函数为y = x * 0.1 + 0.3  
x_data = np.random.rand(100).astype("float32")  
y_data = x_data * 8 + 7
#记录在多次训练中，全局的步数
global_step = tf.Variable(1, name="global_step", trainable=False)

# 指定w和b变量的取值范围（利用TensorFlow来得到w和b的值）  
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0)) #随机生成一个在[-1,1]范围的均匀分布数值  
b = tf.Variable(tf.zeros([1])) #set b=0  
y = W * x_data + b 

# 最小化均方误差  
loss = tf.reduce_mean(tf.square(y - y_data))  
optimizer = tf.train.GradientDescentOptimizer(0.5) #<span style="font-size:14px;">学习率为0.5的梯度下降法</span>  

#除了常规的loss,还有global_step需要更新
train = optimizer.minimize(loss,global_step=global_step)  

saver = tf.train.Saver()
checkpoint_path='C:/Users/TangGuoping/Desktop/check'
CHECKPOINT_FILE = checkpoint_path + "/checkpoint"
LATEST_CHECKPOINT = tf.train.latest_checkpoint(checkpoint_path)
init_op = [
      tf.global_variables_initializer(),
      tf.local_variables_initializer()
  ]
def restore_from_checkpoint(sess, saver, checkpoint):
  if checkpoint:
    print("Restore session from checkpoint: {}".format(checkpoint))
    saver.restore(sess, checkpoint)
    return True
  else:
    print("Checkpoint not found: {}".format(checkpoint))
    return False

with tf.Session() as sess:
    sess.run(init_op)
    restore_from_checkpoint(sess, saver, LATEST_CHECKPOINT)
    gsp=0
    #注意比较local step与global step
    for step in range(7):
        print('before: this run is no.{} local step, is no.{} in global step,  W{} b{}'.format(step,gsp, sess.run(W), sess.run(b)))
        _,gsp=sess.run([train,global_step])  
        print('after: this run is no.{} local step, is no.{} in global step,  W{} b{}'.format(step,gsp, sess.run(W), sess.run(b)))
        if step%3==0:
            saver.save(sess, CHECKPOINT_FILE, global_step=gsp)
    

C:\Users\TangGuoping\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Restore session from checkpoint: C:/Users/TangGuoping/Desktop/check/checkpoint.ckpt-8
INFO:tensorflow:Restoring parameters from C:/Users/TangGuoping/Desktop/check/checkpoint.ckpt-8
before: this run is no.0 local step, is no.0 in global step,  W[6.290342] b[7.916188]
after: this run is no.0 local step, is no.9 in global step,  W[6.376238] b[7.7697845]
before: this run is no.1 local step, is no.9 in global step,  W[6.376238] b[7.7697845]
after: this run is no.1 local step, is no.10 in global step,  W[6.5030117] b[7.731109]
before: this run is no.2 local step, is no.10 in global step,  W[6.5030117] b[7.731109]
after: this run is no.2 local step, is no.11 in global step,  W[6.610241] b[7.6740284]
before: this run is no.3 local step, is no.11 in global step,  W[6.610241] b[7.6740284]
after: this run is no.3 local step, is no.12 in global step,  W[6.7119107] b[7.6257477]
before: this run is no.4 local step, is no.12 in global step,  W[6.7119107] b[7.6257477]
after: this run is no.4 local ste

In [2]:
print('step %dafter%d'%(2,3))

step 2after3


In [3]:
from tensorflow.python import pywrap_tensorflow  
checkpoint_path = 'C:/Users/TangGuoping/Desktop/check'+ "/checkpoint.ckpt"
LATEST_CHECKPOINT = tf.train.latest_checkpoint(checkpoint_path)
reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path)
LATEST_CHECKPOINT = tf.train.latest_checkpoint(checkpoint_path)
var_to_shape_map = reader.get_variable_to_shape_map()
for key in var_to_shape_map:  
    print("tensor_name: ", key)  
    print(reader.get_tensor(key))

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for C:/Users/TangGuoping/Desktop/check/checkpoint.ckpt